In [3]:
import pandas as pd
import numpy as np
from geopy import geocoders
import folium

### Doing some entitiy recognition on the first article of the first newspaper in the first df, using spacy: 

In [2]:
# import spacy
# ! python -m spacy download de_core_news_sm

# nlp = spacy.load("de_core_news_sm")
# doc = nlp(df.iloc[0,13])

# for ent in doc.ents: 
#     """
#     Getting the names of all locations within the loaded newspaper article:
#     """
#     if ent.label_== 'LOC':
#         print(ent.text)

Let's see whether "Widdig" from the above list is really a location:

In [3]:
# import wikipedia as wiki
# wiki.set_lang("de")

# search_string = 'Widdig'
# page = wiki.page(search_string)
# content = page.content
# content[0:680]

### loading pickled dataframes, making new dataframes out of them and saving them to the local drive:

In [4]:
# def pickled_df_loader (year, columns):
#     """
#     loads dataframes from the pickled dfs. 
#     """
#     df= pd.read_pickle(f"./data_deutsches_zeitungsportal_1914_1945/newspapers_ger_{year}_part_1")[columns]
#     df= pd.concat ([df, pd.read_pickle(f"./data_deutsches_zeitungsportal_1914_1945/newspapers_ger_{year}_part_2")[columns]]\
#                    , axis=0)
#     df= pd.concat ([df, pd.read_pickle(f"./data_deutsches_zeitungsportal_1914_1945/newspapers_ger_{year}_part_3")[columns]]\
#                    , axis=0)
    
#     df['publication_date']= df['publication_date'].apply(lambda x: x.strftime("%Y-%m-%d"))
#     df['publication_date'] = pd.to_datetime(df['publication_date'], format='%Y-%m-%d')
    
#     return df

In [5]:
# def challenge_df_generator (begin, end):
#     """
#     loads all pickled dataframes belonging to the years in the year range and returns their concatenation as a single df. 
#     """ 
    
#     columns= ['paper_title', 'publication_date', 'place_of_distribution']
    
#     df= pickled_df_loader (begin, columns)
    
#     for year in range (begin+1, end+1):
#         df= pd.concat([df, pickled_df_loader(year, columns)], axis=0)
        
#     return df

In [6]:
# newspapers_1914_1945_df = challenge_df_generator (1914, 1945)
# newspapers_1914_1945_df.to_pickle("./data_deutsches_zeitungsportal_1914_1945/newspapers_no_article_1914_1945")

### Analyzing the dataframe containing all the data between 1914 and 1945:

In [40]:
newspapers_1914_1945_df= pd.read_pickle(f"./data_deutsches_zeitungsportal_1914_1945/newspapers_no_article_14_45")

In [41]:
newspapers_1914_1945_df.head()

,paper_title,publication_date,place_of_distribution
0,Bensberger Volkszeitung. 1907-1929,1914-02-07,"[Bensberg, Bergisch Gladbach, Bergisch Gladbac..."
1,Bensberger Volkszeitung. 1907-1929,1914-02-07,"[Bensberg, Bergisch Gladbach, Bergisch Gladbac..."
2,Bensberger Volkszeitung. 1907-1929,1914-02-07,"[Bensberg, Bergisch Gladbach, Bergisch Gladbac..."
3,Bensberger Volkszeitung. 1907-1929,1914-02-07,"[Bensberg, Bergisch Gladbach, Bergisch Gladbac..."
4,Der Volksfreund : Tageszeitung für das werktät...,1914-03-04,"[Karlsruhe, Offenburg]"


In [42]:
newspapers_1914_1945_df.tail()

,paper_title,publication_date,place_of_distribution
2056,Neuer hannoverscher Kurier : Nachrichtenblatt ...,1945-10-12,[Hannover]
2057,Neuer hannoverscher Kurier : Nachrichtenblatt ...,1945-10-12,[Hannover]
2058,Neuer hannoverscher Kurier : Nachrichtenblatt ...,1945-10-12,[Hannover]
2059,Olper Kreisblatt : amtliches Mitteilungsblatt ...,1945-12-29,[Olpe]
2060,Olper Kreisblatt : amtliches Mitteilungsblatt ...,1945-12-29,[Olpe]


In [43]:
newspapers_1914_1945_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9204699 entries, 0 to 2060
Data columns (total 3 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   paper_title            object        
 1   publication_date       datetime64[ns]
 2   place_of_distribution  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 280.9+ MB


#### Let's see what we can do with the lists of the cities: 

Extracting all places of distribution and storing them in a pickled python list:

In [11]:
# unique_cities=[]
# for city in newspapers_1914_1945_df['place_of_distribution']:
#     if city not in unique_cities: 
#         unique_cities.append(city)
        
# import pickle
# with open(f"./data_deutsches_zeitungsportal_1914_1945/unique_cities.pkl", "wb") as f:
#     pickle.dump(unique_cities, f)

In [13]:
# city_list= pd.read_pickle(f"./data_deutsches_zeitungsportal_1914_1945/unique_cities.pkl")
# city_list

Extracting names of single cities from the pickled city list: 

In [16]:
# single_city_list= []

# for place in city_list:
#     if isinstance (place, list):
#         for i in range(0, len(place)):
#             if place[i] not in single_city_list:
#                 single_city_list.append(place[i])
#     else: 
#         if place not in city_list:
#             city_list.append(place)

# import pickle
# with open(f"./data_deutsches_zeitungsportal_1914_1945/single_cities.pkl", "wb") as f:
#     pickle.dump(single_city_list, f)

In [ ]:
# cities_1914_1945= pd.read_pickle(f"./data_deutsches_zeitungsportal_1914_1945/single_cities.pkl")
# cities_1914_1945

Pinning the cities from a city list on the world map:

In [6]:
# def map_maker(city_list):
# #plots the cities in the city list on a map.
    
#     city_dict= {}
#     gn= geocoders.GeoNames(username="golisf")
    
#     for city in city_list:
#         if not pd.isna(city):
#             city_dict[city]= gn.geocode(city)
            
#     # Create a map centered at a location, you can adjust the coordinates and zoom level as needed
#     map_center= [51.1657, 10.4515]  # Germany's approximate center
#     my_map= folium.Map(location=map_center, zoom_start=6)
 
#     # Add markers for each city
#     for city in list(city_dict.keys()):
#         folium.Marker(location=city_dict[city], popup=city).add_to(my_map)
        
#     return my_map

In [ ]:
# cities_1914_1945= pd.read_pickle(f"./data_deutsches_zeitungsportal_1914_1945/single_cities.pkl")
# map_maker(cities_1914_1945)

#### Let's see what we can do with the paper titles: 

Extracting the titles of available newspapers and storing them in a pickled python list: 

In [30]:
# unique_papers=[]
# for paper in newspapers_1914_1945_df['paper_title']:
#     if paper not in unique_papers: 
#         unique_papers.append(paper)
        
# import pickle
# with open(f"./data_deutsches_zeitungsportal_1914_1945/unique_papers.pkl", "wb") as f:
#     pickle.dump(unique_papers, f)